In [1]:
%cd /hdd/yuchen

/hdd/yuchen


In [2]:
from PIL import Image
import torch
from torch import nn, optim
import glob
import os
import pandas as pd
import json
import numpy as np
import clip
from torch.utils.data import Dataset, DataLoader, BatchSampler
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import random
from matplotlib.pyplot import imshow
import torchtext
import nltk, re, string, collections
from nltk.util import ngrams
import collections
import scipy 
import IProgress 
from ipywidgets import FloatProgress

%matplotlib inline
BATCH_SIZE = 128
EPOCH = 5

In [3]:
seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.cuda.empty_cache()

In [4]:
save_dir = '/hdd/yuchen/cs292/test/nsdfeat/c/'
files = glob.glob(save_dir + '*')
t = [int(f.split('/')[-1].split('.')[0]) for f in files]
t.sort()
feats_te_avg = []
for file_idx in t: feats_te_avg.append(np.load(f'{save_dir}{file_idx:06}.npy'))
feats_te_avg = np.array(feats_te_avg)

save_dir = '/hdd/yuchen/cs292/train/nsdfeat/c/'
files = glob.glob(save_dir + '*')
t = [int(f.split('/')[-1].split('.')[0]) for f in files]
t.sort()
feats_tr = []
for file_idx in t: feats_tr.append(np.load(f'{save_dir}{file_idx:06}.npy'))
feats_tr = np.array(feats_tr)

In [5]:
feats_tr.shape, feats_te_avg.shape

((8640, 11520), (100, 11520))

In [6]:
from os.path import join as pjoin

betas_csv_dir = '/hdd/yuchen/cs292/fmri/betas_csv/'
sub = '01'
    
data_file = pjoin(betas_csv_dir, f'sub-{sub}_ResponseData.h5')
responses = pd.read_hdf(data_file)
vox_f = pjoin(betas_csv_dir, f'sub-{sub}_VoxelMetadata.csv')
voxdata = pd.read_csv(vox_f)
stim_f = pjoin(betas_csv_dir, f'sub-{sub}_StimulusMetadata.csv')
stimdata = pd.read_csv(stim_f)

train_idx = stimdata[stimdata['trial_type'] == 'train'].index.tolist()
test_idx = stimdata[stimdata['trial_type'] == 'test'].index.tolist()
    
train_fmri = responses[train_idx]
test_fmri = responses[test_idx]
    
train_labels = stimdata[stimdata['trial_type'] == 'train']['stimulus']
test_labels = stimdata[stimdata['trial_type'] == 'test']['stimulus']
    
roi_idx = voxdata[(voxdata['lFFA'] == 1) ]['voxel_id'].tolist()
train_fmri_temp = train_fmri.iloc[roi_idx]
test_fmri_temp = test_fmri.iloc[roi_idx]

roi_neuron = {'lFFA': train_fmri_temp.shape[0]}
for roi in ['VO1', 'VO2', 'TO1', 'TO2',
    'rFFA', 'lPPA', 'rPPA', 
             'lLOC', 'rLOC']:
    roi_idx = voxdata[(voxdata[roi] == 1)]['voxel_id'].tolist()
    train_fmri_temp = pd.concat([train_fmri_temp, train_fmri.iloc[roi_idx]])
    test_fmri_temp = pd.concat([test_fmri_temp, test_fmri.iloc[roi_idx]])
    roi_neuron[roi] = train_fmri.iloc[roi_idx].shape[0]
    
train_fmri = train_fmri_temp
test_fmri = test_fmri_temp

test = stimdata[stimdata['trial_type'] == 'test']
test_fmri_avg = []
test_labels = test['stimulus'].unique()
for stim in test_labels:
    repeated_trial = test[test['stimulus'] == stim].index.tolist()
    test_fmri_avg.append(test_fmri[repeated_trial].mean(axis=1).to_numpy())
test_fmri_avg = np.array(test_fmri_avg)

del responses, voxdata, stimdata
# train_fmri, test_fmri =  train_fmri.to_numpy(), test_fmri.to_numpy()
train_img_label_all, test_img_label_all = train_labels.tolist(), test_labels.tolist()

betas_tr = train_fmri.T
betas_te_avg = test_fmri_avg

In [7]:
roi_neuron

{'lFFA': 154,
 'VO1': 287,
 'VO2': 149,
 'TO1': 369,
 'TO2': 316,
 'rFFA': 399,
 'lPPA': 395,
 'rPPA': 414,
 'lLOC': 1573,
 'rLOC': 1127}

In [8]:
print('feats_tr', feats_tr.shape)
print('betas_tr', betas_tr.shape)

print('feats_te_avg', feats_te_avg.shape)
print('betas_te_avg', betas_te_avg.shape)

feats_tr (8640, 11520)
betas_tr (8640, 5183)
feats_te_avg (100, 11520)
betas_te_avg (100, 5183)


In [9]:
betas_tr = np.array(betas_tr)
betas_te_avg = np.array(betas_te_avg)

betas_tr_ = ((betas_tr - betas_tr.mean()) / betas_tr.std())
betas_te_avg_ = ((betas_te_avg - betas_te_avg.mean()) / betas_te_avg.std())

## baseline

In [10]:
import argparse, os
import numpy as np
from himalaya.backend import set_backend
from himalaya.ridge import RidgeCV
from himalaya.scoring import correlation_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [11]:
alpha = [0.000001,0.00001,0.0001,0.001,0.01, 0.1, 1]

ridge = RidgeCV(alphas=alpha)

preprocess_pipeline = make_pipeline(
        StandardScaler(with_mean=True, with_std=True),
    )
pipeline = make_pipeline(
        preprocess_pipeline,
        ridge,
)    

In [12]:
X = betas_tr
Y = feats_tr
X_te = betas_te_avg
pipeline.fit(X, Y)
scores = pipeline.predict(X_te)
from sklearn.metrics import mean_squared_error
mean_squared_error(scores,feats_te_avg)

1.1444197

In [14]:
np.save('/hdd/yuchen/things_baseline_c.npy',scores)

## text encoder

In [131]:
betas_tr = betas_tr_
betas_te_avg = betas_te_avg_

In [132]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class CustomDataset(Dataset):
    def __init__(self, betas, feats):
        betas = (betas - betas.mean()) / betas.std()
        self.betas = torch.tensor(betas, dtype=torch.float32)
        feats = feats.reshape(-1,15,768)
        feats = feats[:,1:,:].reshape(-1,14*768)
        self.feats = torch.tensor(feats, dtype=torch.float32)

    def __len__(self):
        return len(self.betas)

    def __getitem__(self, idx):
        return {'betas': self.betas[idx], 'feats': self.feats[idx]}

train_dataset = CustomDataset(betas_tr, feats_tr)
test_dataset = CustomDataset(betas_te_avg, feats_te_avg)

In [133]:
cls = feats_tr[0,:768]
cls.shape

(768,)

In [134]:
batch_size = 256  
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for batch in train_dataloader:
    break
num_neurons = batch['betas'].shape[-1]
num_features = batch['feats'].shape[-1]

In [190]:
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self, num_neurons, num_features):
        super(NeuralNetwork, self).__init__()
        self.num_neurons = num_neurons
        channel = 16
        
        self.conv1d = nn.Conv1d(1, channel, kernel_size=1)
        self.relu = nn.GELU()
        
        self.net = nn.Sequential(
            nn.Conv1d(channel, channel, kernel_size=5, padding = 2),
            nn.GELU(),
            nn.Conv1d(channel, channel, kernel_size=5, padding = 2),
            # nn.ReLU(),
            # nn.Conv1d(channel, channel, kernel_size=7, padding=3),
        )

        self.downsample = nn.Conv1d(channel, 1, kernel_size=1)

        self.fc = nn.Sequential(
            nn.Linear(num_neurons, 256),
            nn.GELU(),
            nn.Linear(256, 14*768)
        )

    def forward(self, x):
        x = self.conv1d(x)
        for i in range(2):
            x = x + self.net(x)
        x = self.downsample(x)
        x = x.reshape(-1, self.num_neurons)
        x = self.fc(x)
        # x = x.reshape(-1, 14, 768)
        # x = x.reshape(-1, 14*768)
        return x

In [191]:
# device = 'cuda:1'
# model = NeuralNetwork(num_neurons = num_neurons, num_features = num_features).to(device)
device = 'cuda:1'
model = NeuralNetwork(num_neurons = num_neurons, num_features = num_features).to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100
patience = 5
ct = 0
best_loss_val = np.inf

for epoch in range(num_epochs):
    total_loss_train = 0
    model.train()
    for batch in train_dataloader:
        betas = batch['betas'].to(device).unsqueeze(1)
        feats = batch['feats'].to(device)
        
        pred = model(betas)
        loss = loss_fn(pred, feats)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss_train += loss.item()
    print('[epoch {}] train loss {}'.format(epoch, round(total_loss_train/len(train_dataloader), 5)), end = ' ')

    model.eval()
    total_loss_val = 0
    with torch.no_grad():
        for batch in test_dataloader:
            betas = batch['betas'].to(device).unsqueeze(1)
            feats = batch['feats'].to(device)
            pred = model(betas)
            loss = loss_fn(pred, feats)
            total_loss_val += loss.item()
        total_loss_val = total_loss_val/len(test_dataloader)
        print('val loss {}'.format(round(total_loss_val, 5)))
        
        if total_loss_val < best_loss_val:
            ct = 0
            best_loss_val = total_loss_val
            torch.save(model.state_dict(), '/hdd/yuchen/modelweights_things_text.pth')
        else:
            ct += 1
            if ct > patience:
                print('end')
                break

[epoch 0] train loss 0.88893 val loss 0.82011
[epoch 1] train loss 0.81071 val loss 0.80555
[epoch 2] train loss 0.80685 val loss 0.80564
[epoch 3] train loss 0.8066 val loss 0.80494
[epoch 4] train loss 0.80609 val loss 0.80394
[epoch 5] train loss 0.80337 val loss 0.80133
[epoch 6] train loss 0.79876 val loss 0.79701
[epoch 7] train loss 0.79285 val loss 0.79745
[epoch 8] train loss 0.78576 val loss 0.79409
[epoch 9] train loss 0.77616 val loss 0.80126
[epoch 10] train loss 0.76243 val loss 0.80372
[epoch 11] train loss 0.74555 val loss 0.81167
[epoch 12] train loss 0.72652 val loss 0.82275
[epoch 13] train loss 0.70805 val loss 0.8273
[epoch 14] train loss 0.68966 val loss 0.838
end


In [192]:
model.load_state_dict(torch.load('/hdd/yuchen/modelweights_things_text.pth'))

<All keys matched successfully>

In [193]:
lst = []
lst_gt = []
model.eval()
with torch.no_grad():
    for batch in test_dataloader:
        betas = batch['betas'].to(device).unsqueeze(1)
        feats = batch['feats'].to(device)
        pred = model(betas)
        lst.append(pred.cpu().detach().numpy())
        lst_gt.append(feats.cpu().detach().numpy())
    

In [194]:
# model = NeuralNetwork(num_neurons = num_neurons, num_features = num_features).to(device)
# model.load_state_dict(torch.load('/hdd/yuchen/modelweights_things_text.pth'))

model(betas[0].unsqueeze(0)),model(betas[1].unsqueeze(0))

(tensor([[ 0.3499, -0.1562,  0.7644,  ..., -0.7600,  0.3453, -0.2395]],
        device='cuda:1', grad_fn=<AddmmBackward0>),
 tensor([[-0.2328,  0.0244,  0.9604,  ..., -1.3347,  0.1941, -0.0092]],
        device='cuda:1', grad_fn=<AddmmBackward0>))

In [195]:
lst = np.vstack(lst)
lst_gt = np.vstack(lst_gt)

In [196]:
np.mean(abs(lst-lst_gt),axis=1)

array([0.71181595, 0.73505664, 0.71364105, 0.7019855 , 0.6923032 ,
       0.7086132 , 0.70836276, 0.71297467, 0.7156067 , 0.6969159 ,
       0.6972205 , 0.70788664, 0.7077007 , 0.69142467, 0.68474114,
       0.6840819 , 0.6620259 , 0.6996463 , 0.71602154, 0.70522964,
       0.68986654, 0.66479784, 0.69160914, 0.6791347 , 0.7234187 ,
       0.6783101 , 0.7398947 , 0.72037697, 0.7183888 , 0.6974153 ,
       0.7209481 , 0.69500875, 0.6957981 , 0.6610886 , 0.65416306,
       0.70891315, 0.69346607, 0.6644432 , 0.7043479 , 0.7353466 ,
       0.7088088 , 0.72232044, 0.71578294, 0.68431884, 0.729495  ,
       0.6841668 , 0.6926066 , 0.6670073 , 0.6673077 , 0.6835449 ,
       0.6722989 , 0.7120063 , 0.7248988 , 0.6671987 , 0.6959851 ,
       0.68645775, 0.67412615, 0.6631919 , 0.68711895, 0.71583396,
       0.68420607, 0.7136631 , 0.7173392 , 0.69759357, 0.6986996 ,
       0.6791419 , 0.722405  , 0.7063052 , 0.733666  , 0.68310744,
       0.6859597 , 0.7399479 , 0.7266967 , 0.7144546 , 0.72404

In [197]:
from sklearn.metrics import mean_squared_error
mean_squared_error(lst,lst_gt)

0.794091

In [198]:
lst = lst.reshape(-1, 14, 768)
lst_gt = lst_gt.reshape(-1, 14, 768)
cls_ = np.tile(cls, (100, 1))
cls_ = np.expand_dims(cls_, 1)
lst = np.concatenate((cls_, lst), axis=1)
lst_gt = np.concatenate((cls_, lst_gt), axis=1)
# np.save('/hdd/yuchen/things_temp_lst_c.npy', lst)
np.save('/hdd/yuchen/things_temp_lst_c_again.npy', lst)

In [68]:
np.save('/hdd/yuchen/things_temp_lst_c_again_gt.npy', lst_gt)


## roi

In [82]:
len(roi_neuron)

16

In [148]:
betas_tr_temp = betas_tr_[:, :roi_neuron['lFFA']]
num_feature = 1573
zeros = np.zeros((betas_tr_temp.shape[0], num_feature))
zeros[:, :betas_tr_temp.shape[-1]] = betas_tr_temp
betas_tr_temp = zeros  # (8640, 1573)
for roi in range(2, len(roi_neuron)+1):
    temp = betas_tr_[:, sum(list(roi_neuron.values())[:roi-1]):sum(list(roi_neuron.values())[:roi])]
    #     # if roi == 3: temp = betas_tr_[:, sum(list(roi_neuron.values())[:roi-1]):]
    zeros = np.zeros((betas_tr_.shape[0], num_feature))
    zeros[:, :temp.shape[-1]] = temp
    betas_tr_temp = np.hstack([betas_tr_temp, zeros])

betas_te_avg_temp = betas_te_avg_[:, :roi_neuron['lFFA']]
zeros = np.zeros((betas_te_avg_temp.shape[0], num_feature))
zeros[:, :betas_te_avg_temp.shape[-1]] = betas_te_avg_temp
betas_te_avg_temp = zeros
for roi in range(2, len(roi_neuron)+1):
    temp = betas_te_avg_[:, sum(list(roi_neuron.values())[:roi-1]):sum(list(roi_neuron.values())[:roi])]
    # if roi == 3: temp = betas_te_avg_[:, sum(list(roi_neuron.values())[:roi-1]):]
    zeros = np.zeros((betas_te_avg_.shape[0], num_feature))
    zeros[:, :temp.shape[-1]] = temp
    betas_te_avg_temp = np.hstack([betas_te_avg_temp, zeros])

In [149]:
betas_tr_ = betas_tr_temp
betas_te_avg_ = betas_te_avg_temp

In [150]:
betas_tr_temp.shape

(8640, 25168)

In [152]:
num_roi = 16

num_neurons = betas_tr_.shape[-1]
num_pad = (num_neurons // num_roi + 1) * num_roi - num_neurons

betas_tr = np.pad(betas_tr_, ((0, 0), (0, num_pad)), 'constant', constant_values=0)
betas_te_avg = np.pad(betas_te_avg_, ((0, 0), (0, num_pad)), 'constant', constant_values=0)

In [171]:
cls = feats_tr[0,:768]
cls.shape

(768,)

In [153]:
from PIL import Image
import torch
from torch import nn, optim
import glob
import os
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, BatchSampler
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import random
import nltk, re, string, collections
import scipy 
import IProgress 
from ipywidgets import FloatProgress
from nilearn import connectome
from sklearn.preprocessing import Normalizer, OrdinalEncoder, OneHotEncoder, StandardScaler
from torch.nn import Linear, ReLU, Dropout
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling, BatchNorm, global_mean_pool
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from matplotlib import pyplot as plt

%matplotlib inline

In [160]:
num_neurons_each_roi = int(betas_te_avg.shape[-1]/num_roi)

conn_measure = connectome.ConnectivityMeasure(kind='correlation')

function_con = betas_tr.reshape(-1, num_roi, num_neurons_each_roi)

connectivity = conn_measure.fit_transform([np.mean(function_con, axis=-1)])
for i in range(len(connectivity[0])):
    connectivity[0][i, i] = 0  # setting the i-th row to zero

/hdd/yuchen/anaconda3/envs/ldm/lib/python3.8/site-packages/nilearn/connectome/connectivity_matrices.py:509: FutureWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.13, the default strategy will be replaced by the new strategy and the 'zscore' option will be removed. Please use 'zscore_sample' instead.
  covariances_std = [


In [161]:
connectivity.shape

(1, 16, 16)

In [163]:
adjmatrix = np.where(abs(connectivity[0]) > 0.8)
edge_index = np.array(adjmatrix)  # edge index matrix of shape [2, num_edges]

edge_index.shape

(2, 22)

In [164]:
edge_attr = []
for idx in range(edge_index.shape[1]):
    edge_attr.append(connectivity[0][edge_index[0,idx], edge_index[1,idx]])
edge_attr = np.array(edge_attr)
edge_attr.shape

(22,)

In [158]:
# edge_index_temp = []
# edge_attr_temp = []
# for i in range(edge_attr.shape[0]):
#     edge = edge_index.T[i]
#     if edge[0] < edge[1]:
#         edge_index_temp.append(edge)
#         edge_attr_temp.append(edge_attr[i])
# edge_attr = np.array(edge_attr_temp)
# edge_index = np.array(edge_index_temp).T

In [175]:
feats_tr = feats_tr.reshape(-1,8,768)
feats_tr = feats_tr[:,1:,:].reshape(-1,7*768)
feats_te_avg = feats_te_avg.reshape(-1,8,768)
feats_te_avg = feats_te_avg[:,1:,:].reshape(-1,7*768)

In [176]:
num_trial_tr, num_trials_te = feats_tr.shape[0], feats_te_avg.shape[0]

train_dataset = []

for i in range(num_trial_tr):
    train_dataset.append(Data(x=torch.tensor(betas_tr.reshape(-1, num_roi, num_neurons_each_roi)[i], dtype=torch.float), 
            edge_index=torch.tensor(edge_index, dtype=torch.long), 
            edge_attr=torch.tensor(edge_attr, dtype=torch.float),
            y=torch.tensor(feats_tr[i], dtype=torch.float)
            ))
    
test_dataset = []
for i in range(num_trials_te):
    test_dataset.append(Data(x=torch.tensor(betas_te_avg.reshape(-1, num_roi, num_neurons_each_roi)[i], dtype=torch.float), 
            edge_index=torch.tensor(edge_index, dtype=torch.long), 
            edge_attr=torch.tensor(edge_attr, dtype=torch.float),
            y=torch.tensor(feats_te_avg[i], dtype=torch.float)
            ))

In [177]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [178]:
train_dataset[0], test_dataset[0]

(Data(x=[16, 1574], edge_index=[2, 22], edge_attr=[22], y=[5376]),
 Data(x=[16, 1574], edge_index=[2, 22], edge_attr=[22], y=[5376]))

In [183]:
from torch_geometric.nn import GCNConv, TopKPooling, BatchNorm,ChebConv, SAGEConv,GATConv
from torch_geometric.nn import global_mean_pool, global_max_pool,global_add_pool

class GNN(torch.nn.Module):
    def __init__(self, num_neurons_each_roi):
        super(GNN, self).__init__()
        
        self.conv1 = ChebConv(num_neurons_each_roi, 512, K=4)

        self.conv = ChebConv(512, 512, K=3)
        self.bn = BatchNorm(512)

        nnconv_channel = 4
        self.nnconv = nn.Sequential(
            nn.Conv1d(1, nnconv_channel, kernel_size=1),
            nn.LeakyReLU(),
        )
        self.net = nn.Sequential(
            nn.Conv1d(nnconv_channel, nnconv_channel, kernel_size=3, padding = 1),
            nn.LeakyReLU(),
        )
        self.downsample = nn.Conv1d(nnconv_channel, 1, kernel_size=1)

        self.fc = nn.Sequential(
            nn.Linear(512, 512),
            nn.LeakyReLU(),
            nn.Linear(512, 7*768)
        )

    def forward(self, data):
        x, edge_index, batch,edge_attr = data.x, data.edge_index, data.batch, data.edge_attr
        batch_size = batch.max() + 1

        x = F.relu(self.conv1(x, edge_index, edge_attr))
        
        for i in range(1):
            a = self.conv(x, edge_index, edge_attr)
            a = self.bn(a)
            x = x + F.leaky_relu(a)
        
        x = global_mean_pool(x, batch) # print(x.shape) torch.Size([64, 256])

        x = x.unsqueeze(1)
        # x = self.nnconv(x)
        # for j in range(2):
        #     x = x + self.net(x)
        # x = self.downsample(x)
        x = x.flatten(start_dim=1)
        x = self.fc(x)
        
        return x

In [184]:
device = 'cuda:1'
model = GNN(num_neurons_each_roi=num_neurons_each_roi).to(device)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

num_epochs = 100
patience = 5
ct = 0
best_loss_val = np.inf

for epoch in range(num_epochs):
    total_loss_train = 0
    model.train()
    for batch in train_dataloader:
        batch = batch.to(device)
        batch_num = batch.batch.max() + 1
        feats = batch['y'].reshape(batch_num,-1).to(device)
        pred = model(batch)
        
        loss = loss_fn(pred, feats)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss_train += loss.item()
    print('[epoch {}] train loss {}'.format(epoch, round(total_loss_train/len(train_dataloader), 5)), end = ' ')

    model.eval()
    total_loss_val = 0
    with torch.no_grad():
        for batch in test_dataloader:
            batch = batch.to(device)
            batch_num = batch.batch.max() + 1
            feats = batch['y'].reshape(batch_num,-1).to(device)
            pred = model(batch)
            loss = loss_fn(pred, feats)
            total_loss_val += loss.item()
        total_loss_val = total_loss_val/len(test_dataloader)
        print('val loss {}'.format(round(total_loss_val, 5)))
        
        if total_loss_val < best_loss_val:
            ct = 0
            best_loss_val = total_loss_val
            torch.save(model.state_dict(), '/hdd/yuchen/modelweights_things_text_gcn_roi.pth')
        else:
            ct += 1
            if ct > patience:
                print('end')
                break

[epoch 0] train loss 0.76835 val loss 0.69196
[epoch 1] train loss 0.68776 val loss 0.69045
[epoch 2] train loss 0.686 val loss 0.68925
[epoch 3] train loss 0.68339 val loss 0.68794
[epoch 4] train loss 0.67789 val loss 0.68769
[epoch 5] train loss 0.66863 val loss 0.68519
[epoch 6] train loss 0.65451 val loss 0.68561
[epoch 7] train loss 0.63741 val loss 0.6892


KeyboardInterrupt: 